In [ ]:
!pip install boto3 awscli


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 29.5 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 

In [ ]:
import getpass

# Input AWS credentials and region
aws_access_key_id = getpass.getpass('Enter AWS Access Key ID: ')
aws_secret_access_key = getpass.getpass('Enter AWS Secret Access Key: ')
aws_region = input('Enter AWS Region: ')

!aws configure set aws_access_key_id $aws_access_key_id
!aws configure set aws_secret_access_key $aws_secret_access_key
!aws configure set default.region $aws_region


Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········
Enter AWS Region: us-east-1


In [ ]:
stream_name = 'Fxweatherstream'


In [ ]:
import boto3
import json
import time
# Initialize Kinesis client
kinesis_client = boto3.client('kinesis', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=aws_region)


In [ ]:

def get_shard_iterator():
    response = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId='shardId-000000000003',  # Replace with your shard ID
        ShardIteratorType='TRIM_HORIZON'
    )
    return response['ShardIterator']

In [ ]:
def get_records(shard_iterator):
    response = kinesis_client.get_records(
        ShardIterator=shard_iterator,
        Limit=10
    )
    return response.get('Records', []), response.get('NextShardIterator')

In [ ]:
def process_weather_data(data):
    print(f"Received Weather Data: {data}")

In [ ]:
if __name__ == "__main__":
    shard_iterator = get_shard_iterator()

    while True:
        records, next_shard_iterator = get_records(shard_iterator)
        for record in records:
            data = json.loads(record['Data'])
            process_weather_data(data)

        if not next_shard_iterator:
            print("No more records in the shard. Exiting.")
            break

        shard_iterator = next_shard_iterator
        time.sleep(2)  # Add a delay between reads (adjust as needed)


Received Weather Data: {'city': 'Los Angeles', 'temperature': 24.74, 'humidity': 62.39, 'timestamp': 1704025805}
Received Weather Data: {'city': 'Los Angeles', 'temperature': 10.31, 'humidity': 61.62, 'timestamp': 1704025809}
Received Weather Data: {'city': 'Los Angeles', 'temperature': 19.4, 'humidity': 47.03, 'timestamp': 1704025822}
Received Weather Data: {'city': 'Los Angeles', 'temperature': 17.46, 'humidity': 65.6, 'timestamp': 1704025825}
Received Weather Data: {'city': 'Los Angeles', 'temperature': 22.99, 'humidity': 79.78, 'timestamp': 1704025827}
Received Weather Data: {'city': 'Los Angeles', 'temperature': 12.32, 'humidity': 70.8, 'timestamp': 1704025831}


KeyboardInterrupt: ignored

In [ ]:
!aws kinesis delete-stream --stream-name Fxweatherstream
